In [1]:
import sqlite3
import time
import pandas

In [2]:
# Functions to read data and connect to database

# Read data from Pandas 
def read_data_pandas():
    start_time = time.time()
    df = pandas.read_csv("dataset.csv", low_memory=False)
    end_time = time.time()
    return df, end_time - start_time

# Create SQLite client
def connect_database():
    start_time = time.time()
    conn = sqlite3.connect('dataset.db')
    end_time = time.time()
    return conn, end_time - start_time

In [4]:
# Reading data and connecting to database:
print("\nReading dataset.csv into Pandas data frame...")
df,t = read_data_pandas()
print("Time to load CSV dataset to Pandas: %s seconds", t)
print("\nConnecting SQLite client to dataset.db...")
conn,t = connect_database()
print("Time to connect to SQLite database: %s seconds", t)


Reading dataset.csv into Pandas data frame...
Time to load CSV dataset to Pandas: %s seconds 38.25836706161499

Connecting SQLite client to dataset.db...
Time to connect to SQLite database: %s seconds 0.005471229553222656


Pandas copies all data over to memory and to a Data Frame format. On the other hand, SQLite keeps the data in disk and queries using a client.  Data transfer to memory, depending on the size of the data, can take significantly more time than keeping data in a database. 

For running ad-hoc queries; it is often more practical to keep data in a database. 

In [5]:
# Drop rows where Make is NaN
# Pandas: 
df = df[~df["Make"].isna()]
df["Make"].count()

4357544

In [6]:
# Delete rows where Make is empty
# SQLite:
conn.cursor().execute('DELETE FROM citations WHERE Make == ""')
conn.commit()

In [7]:
# Query a) calculate top 25 most common 'makes'

# Using Pandas:
def query_a_pandas(df, print_result=False):
    start_time = time.time()
    rows = df['Make'].value_counts()[:25].index.tolist()    
    end_time = time.time()
    if(print_result):
        for row in rows:
            print(row)
    return end_time - start_time

# Using SQLite:
def query_a_sqlite(conn, print_result=False):
    start_time = time.time()
    cursor = conn.execute('SELECT Make, count(*) as number_of_citations FROM citations GROUP BY Make ORDER BY number_of_citations DESC LIMIT 25')
    rows = cursor.fetchall()
    end_time = time.time()
    if(print_result):
        for row in rows:
            print(row)
    return end_time - start_time

print("Query Executions:")
exe_times_sqlite=[]
exe_times_pandas=[]
for i in range(3):
    ts = query_a_sqlite(conn)
    tp = query_a_pandas(df)
    exe_times_sqlite.append(ts)
    exe_times_pandas.append(tp)
    print("SQLite Query run "+str(i+1)+": "+str(ts)+" seconds")
    print("Pandas Query run "+str(i+1)+": "+str(tp)+" seconds")
print("-----------------")
print("Average run time:")
print("SQLite: "+str(sum(exe_times_sqlite)/len(exe_times_sqlite)))
print("Pandas: "+str(sum(exe_times_pandas)/len(exe_times_pandas)))
print()


Query Executions:
SQLite Query run 1: 2.469014883041382 seconds
Pandas Query run 1: 0.406688928604126 seconds
SQLite Query run 2: 2.582409143447876 seconds
Pandas Query run 2: 0.3519001007080078 seconds
SQLite Query run 3: 2.900399923324585 seconds
Pandas Query run 3: 0.37567830085754395 seconds
-----------------
Average run time:
SQLite: 2.6506079832712808
Pandas: 0.37808911005655926



In [8]:
# Query b) calculate most common 'Color' for each 'Make'

# Using Pandas:
def query_b_pandas(df, print_result=False):
    start_time = time.time()
    df = df.dropna(subset=['Color'])
    rows = df.groupby(['Make'])['Color'].agg(lambda x: pandas.Series.mode(x)[0]).to_dict()
    end_time = time.time()
    if(print_result):
        for row in rows:
            print(row, rows[row])            
    return end_time - start_time

# Using SQLite:
def query_b_sqlite(conn, print_result=False):
    start_time = time.time()
    cursor = conn.execute('''
        SELECT Make, Color FROM (
            SELECT 
                ROW_NUMBER() OVER(PARTITION BY Make) AS RowNumber,
                Make, 
                Color, 
                count(*) as number_of_citations 
            FROM 
                citations 
            WHERE 
                Color != "" 
            GROUP BY 
                Make, 
                Color 
            ORDER BY 
                Make, 
                number_of_citations DESC, 
                Color) as A 
        WHERE RowNumber = 1
    ''')
    rows = cursor.fetchall()
    end_time = time.time()
    if(print_result):
        for row in rows:
            print(row)
    return end_time - start_time

print("Query Executions:")
exe_times_sqlite=[]
exe_times_pandas=[]
for i in range(3):
    ts = query_b_sqlite(conn)
    tp = query_b_pandas(df)
    exe_times_sqlite.append(ts)
    exe_times_pandas.append(tp)
    print("SQLite Query run "+str(i+1)+": "+str(ts)+" seconds")
    print("Pandas Query run "+str(i+1)+": "+str(tp)+" seconds")
print("-----------------")
print("Average run time:")
print("SQLite: "+str(sum(exe_times_sqlite)/len(exe_times_sqlite)))
print("Pandas: "+str(sum(exe_times_pandas)/len(exe_times_pandas)))
print()



Query Executions:
SQLite Query run 1: 4.300311088562012 seconds
Pandas Query run 1: 4.417699813842773 seconds
SQLite Query run 2: 5.368381023406982 seconds
Pandas Query run 2: 1.9101898670196533 seconds
SQLite Query run 3: 5.2095417976379395 seconds
Pandas Query run 3: 2.0750911235809326 seconds
-----------------
Average run time:
SQLite: 4.9594113032023115
Pandas: 2.8009936014811196



In [9]:
# Query c) find the first ticket issued for each 'Make'

# Using Pandas
def query_c_pandas(df, print_result=False):
    start_time = time.time()
    df = df.dropna(subset=['Issue Date'])
    rows = df.groupby('Make')['Issue Date'].agg('min').to_dict()
    end_time = time.time()
    if(print_result):
        for row in rows:
            print(row, rows[row])
    return end_time - start_time

# Using SQLite
def query_c_sqlite(conn, print_result=False):
    start_time = time.time()
    cursor = conn.execute('Select Make, min("Issue Date") from citations group by Make')
    rows = cursor.fetchall()
    end_time = time.time()
    if(print_result):
        for row in rows:
            print(row)
    return end_time - start_time

print("Query Executions:")
exe_times_sqlite=[]
exe_times_pandas=[]
for i in range(3):
    ts = query_c_sqlite(conn)
    tp = query_c_pandas(df)
    exe_times_sqlite.append(ts)
    exe_times_pandas.append(tp)
    print("SQLite Query run "+str(i+1)+": "+str(ts)+" seconds")
    print("Pandas Query run "+str(i+1)+": "+str(tp)+" seconds")
print("-----------------")
print("Average run time:")
print("SQLite: "+str(sum(exe_times_sqlite)/len(exe_times_sqlite)))
print("Pandas: "+str(sum(exe_times_pandas)/len(exe_times_pandas)))
print()

Query Executions:
SQLite Query run 1: 3.2358012199401855 seconds
Pandas Query run 1: 5.491661071777344 seconds
SQLite Query run 2: 3.4396870136260986 seconds
Pandas Query run 2: 1.8667690753936768 seconds
SQLite Query run 3: 2.852605104446411 seconds
Pandas Query run 3: 1.7528140544891357 seconds
-----------------
Average run time:
SQLite: 3.1760311126708984
Pandas: 3.0370814005533853



Pandas vs. SQLite (Run Time)
There are two components involved: Loading Data, Running Queries
Loading Data: 
As stated above, Panda loads data in memory. Depending on the size of the data this may take a long time. However, connecting to database that stores data in disk is almost instant.
Running Queries: 
Panda's ability to keep data in Memory means fast queries. Data indexing in Pandas is not as efficient as a SQL based database like SQLite. Despite that, we still see Pandas queries run faster than SQL.
Conclusion: 
Running ad-hoc queries are probably more efficient in SQL as we do not have to load the entire dataset in Memory. 
For an always running server that has the data loaded in memory, Pandas seems to be faster in response time. It is also important to note that the performance of the SQL queries depends largely on the I/O power of the computer where they are running.

Ease of Use
Both SQL and Pandas are powerful, well documented, and popular technologies that can address a wide range of query types. Pandas keep data formats close to Python data types which make it a little bit easier for a developer to handle. However, generally speaking, they are both easy technologies to query large datasets. 


In [12]:
# Part 2 - Second Question: 
# Is an out-of-state license plate more likely to be expired at the time of recieving a 
# ticket than an in-state license plate?

# It appears that the dataset is from the state of California. 
# If this assumption turned out to be wrong, we could use the following 
# library to get the state where the citation was given using the
# coordinates recorded at the time of citations. 
# This will take hours, if not days, to get the state for all data. 

import reverse_geocoder as rg
coordinates = (37.38605,-122.08385) #just an example for the sake of demo
results = rg.search(coordinates) 
print(results[0]['admin1'])

# To save time this exercise assumes all citations occured in California

Loading formatted geocoded file...
California


In [50]:
# Separate out of state autos from CA autos
non_ca_cars = df[df['RP State Plate'] != 'CA']
# Drop extra columns
non_ca_cars = non_ca_cars.drop(columns=['Ticket number', 'Issue time', 'Meter Id', 
                                        'Marked Time', 'VIN', 'Make', 'Body Style', 
                                        'Color', 'Location', 'Route', 'Agency', 'Violation code', 
                                        'Violation Description', 'Fine amount', 'Latitude', 'Longitude'])

non_ca_cars = non_ca_cars.dropna(subset=['Plate Expiry Date'])
non_ca_cars.loc[:,"Plate Expiry Date"] = non_ca_cars["Plate Expiry Date"].astype(int)
non_ca_cars.loc[:,"Plate Expiry Date"] = non_ca_cars["Plate Expiry Date"].astype(str)

# Re-format all dates to match
non_ca_cars.loc[:,'Plate Expiry Date Formatted'] = pandas.to_datetime(
    non_ca_cars['Plate Expiry Date'], format='%Y%m', errors='coerce').dropna()


non_ca_cars.loc[:,'Issue Date Formatted'] = pandas.to_datetime(
    non_ca_cars['Issue Date'], format='%Y-%m', errors='coerce').dropna()

# Again, drop extra columns
non_ca_cars = non_ca_cars.drop(columns=['Plate Expiry Date', 'Issue Date'])

expired_plates = len(non_ca_cars[non_ca_cars['Issue Date Formatted'] > non_ca_cars['Plate Expiry Date Formatted']])

not_expired_plates = len(non_ca_cars[non_ca_cars['Issue Date Formatted'] < non_ca_cars['Plate Expiry Date Formatted']])

print("Percentage of out of state cars that received a citation while their license plate was expired:")
print(str(expired_plates/(expired_plates+not_expired_plates)*100))

Percentage of out of state cars that received a citation while their license plate was expired:
23.49205627656431


In [52]:
# And now, the in state cars:

ca_cars = df[df['RP State Plate'] == 'CA']
ca_cars['Ticket number'].count()

# Drop extra columns
ca_cars = ca_cars.drop(columns=['Ticket number', 'Issue time', 'Meter Id', 'Marked Time', 'VIN', 'Make', 'Body Style', 'Color', 'Location', 'Route', 'Agency', 'Violation code', 'Violation Description', 'Fine amount', 'Latitude', 'Longitude'])

ca_cars = ca_cars.dropna(subset=['Plate Expiry Date'])
ca_cars.loc[:,"Plate Expiry Date"] = ca_cars["Plate Expiry Date"].astype(int)
ca_cars.loc[:,"Plate Expiry Date"] = ca_cars["Plate Expiry Date"].astype(str)


# Re-format all dates to match
ca_cars.loc[:,'Plate Expiry Date Formatted'] = pandas.to_datetime(
    ca_cars['Plate Expiry Date'], format='%Y%m', errors='coerce').dropna()

ca_cars.loc[:,'Issue Date Formatted'] = pandas.to_datetime(
    ca_cars['Issue Date'], format='%Y-%m', errors='coerce').dropna()

# Again, drop extra columns
ca_cars = ca_cars.drop(columns=['Plate Expiry Date', 'Issue Date'])

expired_plates = len(ca_cars[ca_cars['Issue Date Formatted'] > ca_cars['Plate Expiry Date Formatted']])

not_expired_plates = len(ca_cars[ca_cars['Issue Date Formatted'] < ca_cars['Plate Expiry Date Formatted']])

print("Percentage in state cars that received a citation while their license plate was expired:")
print(str(expired_plates/(expired_plates+not_expired_plates)*100))

Percentage in state cars that received a citation while their license plate was expired:
22.02060669282623
